# Play Game

In this fle you can try playing against every model we have created.

In [7]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

In [16]:
def print_board(board):
    """Display the Connect-4 board."""
    symbols = {1: "X", -1: "O", 0: "."}
    for row in board:
        print(" ".join(symbols[cell] for cell in row))
    print("\n")

def is_valid_move(board, col):
    """Check if a move is valid."""
    return board[0, col] == 0

def make_move(board, col, player):
    """Place a player's piece in the chosen column."""
    for row in range(5, -1, -1):  # Start from the bottom of the column
        if board[row, col] == 0:
            board[row, col] = player
            return

def get_bot_move(model, board):
    """Get the bot's move using any model."""
    # Determine model input shape
    input_shape = model.input_shape[1:]  # Exclude batch dimension

    if input_shape == (6, 7, 1):  # CNN input format
        input_board = board.reshape(1, 6, 7, 1)
    elif input_shape == (42,):  # MLP input format
        input_board = board.flatten().reshape(1, -1)
    else:
        raise ValueError(f"Unsupported model input shape: {input_shape}")

    # Predict and mask invalid moves
    predictions = model.predict(input_board)[0]
    valid_columns = [col for col in range(7) if is_valid_move(board, col)]
    masked_predictions = np.array([-1 if col not in valid_columns else predictions[col] for col in range(7)])

    # Choose the column with the highest valid probability
    return np.argmax(masked_predictions)

def is_winner(board, player):
    """Check if a player has won."""
    # Check horizontal, vertical, and diagonal (both directions) wins
    for row in range(6):
        for col in range(7):
            if (
                col + 3 < 7 and all(board[row, col + i] == player for i in range(4))
                or row + 3 < 6 and all(board[row + i, col] == player for i in range(4))
                or row + 3 < 6 and col + 3 < 7 and all(board[row + i, col + i] == player for i in range(4))
                or row + 3 < 6 and col - 3 >= 0 and all(board[row + i, col - i] == player for i in range(4))
            ):
                return True
    return False

def play_game(model_path):
    """Play a game of Connect-4 against the bot."""
    model = load_model(model_path)  # Load the trained model
    board = np.zeros((6, 7), dtype=int)  # Initialize an empty board
    print("Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).\n")
    print_board(board)
    while True:
        # Player's move
        while True:
            try:
                col = int(input("Enter your move (0-6): "))
                if 0 <= col <= 6 and is_valid_move(board, col):
                    break
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Please enter a number between 0 and 6.")
        make_move(board, col, 1)
        print("Your move:")
        print_board(board)
        if is_winner(board, 1):
            print("Congratulations! You win!")
            break

        # Check for a draw
        if not any(board[0, :] == 0):
            print("It's a draw!")
            break

        # Bot's move
        bot_col = get_bot_move(model, board)
        make_move(board, bot_col, -1)
        print(f"Bot chooses column {bot_col}:")
        print_board(board)
        if is_winner(board, -1):
            print("Bot wins! Better luck next time!")
            break

def play_game_players(player1, player2):
    """
        params:
            player1: Player, will always move first
            player2: Player
        Plays the game using the provided player objects. 
        Player1 and Player2 should have different markers

        Returns: The marker of the player who won, or 0 for draw
    """
    if player1.marker == player2.marker:
        print("ERROR! Players must have different markers!")
        return
        
    board = np.zeros((6, 7), dtype=int)  # Initialize an empty board
    print_board(board)

    while True:
        # Player's move
        player1Move = player1.GetMove(board)
        make_move(board, player1Move, player1.marker)
        # print("Your move:")
        print_board(board)
        if is_winner(board, player1.marker):
            print("Congratulations! Player 1 wins!")
            return player1.marker

        # Check for a draw
        if not any(board[0, :] == 0):
            print("It's a draw!")
            return 0

        # Bot's move
        player2Move = player2.GetMove(board)
        make_move(board, player2Move, player2.marker)
        # print(f"Bot chooses column {bot_col}:")
        print_board(board)
        if is_winner(board, player2.marker):
            print("Player 2 wins! Better luck next time!")
            return player2.marker
    

In [17]:
# player classes
import copy

class Player:
    def __init__(self, marker):
        self.marker = marker

class RandomPlayer(Player):
    def GetMove(self, board):
        available = [col for col, value in enumerate(board[0]) if value == 0]
        return np.random.choice(available)

class SophisticatedPlayer(Player):
    def GetMove(self, board):
        available = [col for col, value in enumerate(board[0]) if value == 0]

        # get winning move if it exists
        for move in available:
            temp_board = copy.deepcopy(board)
            make_move(temp_board, move, self.marker)
            if is_winner(temp_board, self.marker):
                print("Winning!")
                return move

        # block winning move for opponents
        for move in available:
            temp_board = copy.deepcopy(board)
            make_move(temp_board, move, -1) # opponent's marker
            if is_winner(temp_board, -1):
                print("Blocking!")
                return move

        # return a random move if no other criteria are met
        print("Choosing random")
        return np.random.choice(available)

class RealPlayer(Player):
    def GetMove(self, board):
        print("Your move:")
        while True:
            try:
                col = int(input("Enter your move (0-6): "))
                if 0 <= col <= 6 and is_valid_move(board, col):
                    return col
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Please enter a number between 0 and 6.")

class H5Player(Player):
    def __init__(self, marker, model_path):
        Player.__init__(self, marker)
        self.model = load_model(model_path)

    def GetMove(self, board):
        return get_bot_move(self.model, board)

class PKLPlayer(Player):
    def __init__(self, marker, model_path):
        Player.__init__(self, marker)
        self.model = joblib.load(model_path)

    def GetMove(self, board):
        return self.model.predict(board.reshape(1, -1))[0]


board = np.zeros((6, 7), dtype=int)
player = SophisticatedPlayer(1)

print(player.GetMove(board))
board[5, 0] = -1
board[4, 1] = -1
board[3, 2] = -1
board[5, 3] = 1
board[4, 3] = 1
board[3, 3] = 1
print(board)
print(player.GetMove(board))

Choosing random
1
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0 -1  0  1  0  0  0]
 [-1  0  0  1  0  0  0]]
Winning!
3


# Play against CNN Model

In [ ]:
# play_game("connect4_cnn_model.h5")

# this is how we should do plays from now on
# the player who is passed in first will always go first
# the markers for each player need to be different
# and should be either -1 or 1
# 
# this should allow us to easily test the models in 
# actual play scenarios over many games against different strategies
realPlayer = RealPlayer(1)
CNNPlayer = H5Player(-1, "connect4_cnn_model.h5")
print(play_game_players(realPlayer, CNNPlayer))

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .


Your move:


Enter your move (0-6):  0


. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .


Your move:


Enter your move (0-6):  0


. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .


Your move:


# Play Against MLP Model

In [15]:
play_game("connect4_mlp_model.h5")

Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Bot chooses column 0:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
O . . X . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Bot chooses column 0:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .
O . . X . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
O . . X . . .
O . . X . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Bot chooses column 0:
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .
O . . X . . .
O . . X . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. .

# Play Against Decision Tree Model

In [18]:
import numpy as np


def is_valid_move(board, col):
    """Check if a move is valid."""
    return board[0, col] == 0

def make_move(board, col, player):
    """Make a move on the board."""
    for row in range(5, -1, -1):
        if board[row, col] == 0:
            board[row, col] = player
            break

def is_winner(board, player):
    """Check if the player has won."""
    for row in range(6):
        for col in range(7):
            if (
                col + 3 < 7 and all(board[row, col + i] == player for i in range(4)) or
                row + 3 < 6 and all(board[row + i, col] == player for i in range(4)) or
                row + 3 < 6 and col + 3 < 7 and all(board[row + i, col + i] == player for i in range(4)) or
                row + 3 < 6 and col - 3 >= 0 and all(board[row + i, col - i] == player for i in range(4))
            ):
                return True
    return False

def print_board(board):
    """Print the Connect-4 board."""
    symbols = {1: "X", -1: "O", 0: "."}
    for row in board:
        print(" ".join(symbols[cell] for cell in row))
    print("\n")

def play_game(model_path):
    """Play a game of Connect-4 against the bot."""
    model = joblib.load(model_path)  # Load the trained model
    board = np.zeros((6, 7), dtype=int)  # Initialize an empty board
    print("Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).\n")
    print_board(board)
    while True:
        # Player's move
        while True:
            try:
                col = int(input("Enter your move (0-6): "))
                if 0 <= col <= 6 and is_valid_move(board, col):
                    break
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Please enter a number between 0 and 6.")
        make_move(board, col, 1)
        print("Your move:")
        print_board(board)
        if is_winner(board, 1):
            print("Congratulations! You win!")
            break

        # Bot's move
        # Assuming the model takes the board as input and outputs the best column to play
        bot_move = model.predict(board.reshape(1, -1))[0]

        make_move(board, bot_move, -1)
        print("Bot's move:")
        print_board(board)
        if is_winner(board, -1):
            print("Bot wins! Better luck next time.")
            break

# Example usage
# play_game('decision_tree_model.pkl')

In [19]:
play_game('decision_tree_model.pkl')

Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .


Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .


Bot's move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .


Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X X . . . . .


Bot's move:
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
X X . . . . .


Your move:
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O X . . . . .
X X . . . . .


Bot's move:
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
O X . . . . .
X X . . . . .


Your move:
. . . . . . .
X . . . . . .
O . . . . . .
O . . . . . .
O X . . . . .
X X . . . . .


Bot's move:
O . . . . . .
X . . . . . .
O . . . . . .
O . . . . . .
O X . . . . .
X X . . . . .


Your move:
O . . . . . .
X . . . . . .
O . . . . . .
O X . . . . .
O X . . . 

# Play against SVM Model

In [22]:
play_game('svm_model.pkl')

Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .




Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .


Bot's move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . X . .


Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X X . .


Bot's move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . X X . .


Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . X X X . .


Bot's move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O O X X X . .


Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O O X X X X .


Congratulations! You win!
